# Moving Average

_Moving Average_ (MA) is an indicator that constantly describes the average price over a period. It's also called _rolling average_. The mean over the last $k$ data points can be calculated as:

$\displaystyle {\begin{aligned}{\textit {SMA}}_{k}={\frac {p_{n-k+1}+p_{n-k+2}\cdots +p_{n}}{k}}={\frac {1}{k}}\sum _{i=n-k+1}^{n}p_{i}\end{aligned}}$

Intuitively, the value for a 3 days moving average in a given date $t$ is computed by simply calculating the average of the previous 3 values: $(t + t_{-1} + t_{-2}) / 3$.

Luckily, Pandas offers us the `.rolling()` function, that accepts the number of days to use:

In [ ]:
import pandas as pd
from sample_data import stock_prices

msft = stock_prices('msft')
ma50 = msft.Close.rolling(50).mean()
ma200 = msft.Close.rolling(200).mean()

`ma50` is a Series with all the computed moving average values

In [ ]:
ma50.tail(4)

Using `ma50` and `ma200`, we can plot them to observe the trends over time. Lines should follow the main price values, but you should observe a more _soft_ reactions to price changes.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['font.family'] = "monospace"

ig, ax = plt.subplots(figsize=(10,6))

msft.Close['2019':].plot(color='#D1D5DB', label="Price")
ma50['2019':].plot(color='#3B82F6', label="MA50")
ma200['2019':].plot(color='#1E3A8A', label="MA200")
plt.legend()
plt.show()

### Exercise 5.1

Read 2019 data for `aapl` and compute the moving average for the last 50 days using Spark.

### Exercise 5.2

Compute the same MA20, but this time by using [the Pandas API for Spark](https://databricks.com/blog/2021/10/04/pandas-api-on-upcoming-apache-spark-3-2.html). 

There could be a warning message like this one: _No Partition Defined for Window operation_. What does it mean?

## Exponential Moving Average (EMA)

A variation of this is to use the _Exponential Moving Average (EMA)_, that gives more weight to recent prices. SMA represent the average of prices in a timespan, and are better suited to identify "resistance" levels. In contrast, EMA has less lag and is therefore more sensitive to recent prices.

${ewm}(α)=(1−α)⋅{emw}_{t-1}+α⋅x_t$

Where $x_t$ is the price value as of today and ${emw}_{t-1}$ the EMA value from yesterday.

EMA is calculated using a smoothing or decay factor $\alpha$ that determines the weight given to the current period's value. The smoothing factor is a value between 0 and 1. A smaller smoothing factor gives more weight to recent data, making the EMA more responsive to recent price changes. Conversely, a larger smoothing factor places more weight on older data, resulting in a smoother and slower-moving EMA.

$\alpha$ is typically calculated as $\alpha = 2 / (N + 1)$, where $N$ represents the number of periods or time intervals used in the EMA calculation.

Pandas offers us the [ewm](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html) function to calculate the exponentially weighted windows:

In [ ]:
msft_2016 = msft['2016':'2017']
ema10 = msft_2016.Close.ewm(span=10, adjust=False).mean()
sma50 = msft_2016.Close.rolling(50).mean()
sma100 = msft_2016.Close.rolling(100).mean()

ig, ax = plt.subplots(figsize=(10,6))

msft_2016.Close.plot(ax=ax, color='#D1D5DB', label="price")
ema10.plot(ax=ax, color='#EF4444', linestyle="--", label="ema10")
sma50.plot(ax=ax, color='#3B82F6', label="sma50")
sma100.plot(ax=ax, color='#1E3A8A', label="sma100")
plt.legend()
plt.show()

One interesting thing about `ewm` is that it supports incremental updates. To use it, you have to call the `online` method right after `ewm` and that would return an object that allows to update the data with new data points.

In [ ]:
online_ema10 = msft_2016.Close.ewm(span=10, adjust=False).online()
online_ema10.mean().tail(2)

In [ ]:
online_ema10.mean(update=pd.Series([86.0]))

## MACD

There's another indicator called _Moving Average Convergence/Divergence Oscillator (MACD)_. It's calculated by getting two moving averages and substracting the longer moving average by the shorter one. In the calculated MCDA below, negative MACD values indicate that the 12-day EMA is below the 26-day EMA.

In [ ]:
ema12 = msft_2016.Close.ewm(span=12, adjust=False).mean()
ema26 = msft_2016.Close.ewm(span=26, adjust=False).mean()
macd = ema12 - ema26
macd

Additionally, we can calculate a _signal line_ that would identify events on the MACD when both lines cross. When signal line is below the MACD, things are improving over time.

In [ ]:
signal = macd.ewm(span=9, adjust=False).mean()

fig, ax = plt.subplots(figsize=(10,6))
msft_2016.Close.plot(ax=ax, color='#374151', alpha=0.2, secondary_y=True)
macd.plot(ax=ax, color='blue')
signal.plot(ax=ax, color='red')